In [ ]:
import os
import pickle
import pandas as pd
import numpy as np
from operator import itemgetter

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

import tensorflow.compat.v1 as tf
from tensorflow import keras
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, BatchNormalization
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras import regularizers, optimizers

from sklearn.model_selection import KFold
from sklearn.neighbors import KNeighborsClassifier

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

import time

In [ ]:
def unpickle() :
    for file in os.listdir("datataset") :
        if file == "batches.meta" :
            continue
        file_name = "datataset/" + file
        with open(file_name, 'rb') as fo :
            dict = pickle.load(fo, encoding='latin1')
        data = np.array([dict['data']])
        data = data.reshape((len(dict['data']), 3, 32, 32)).transpose(0, 2, 3, 1)
        label = [dict['labels']]

        return data, label

In [ ]:
def load_label_names():
    return ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

In [ ]:
def vision_data() :
    features, labels = unpickle()
    label_names = load_label_names()

    label_counts = dict(zip(*np.unique(labels, return_counts=True)))
    data_list = []
    num = 6000
    for key, value in label_counts.items():
        print('Label Counts of [{}]({}) : {}'.format(key, label_names[key].upper(), num))
        data_list.append([num, key])


    data_list = sorted(data_list, key=itemgetter(0))
    features_sorted = []
    importance_sorted = []
    for i in data_list :
        features_sorted += [i[1]]
        importance_sorted += [i[0]]
#     print(len(importance_sorted), importance_sorted, features_sorted)

    p1 = plt.barh(range(len(importance_sorted)), importance_sorted)
    plt.bar_label(p1)
    plt.title("Distribution des données dans chaque classe")
    plt.yticks(range(len(importance_sorted)), features_sorted)


In [ ]:
vision_data()

In [ ]:
# features, labels = read_data(label_ids=label_ids)
features, labels = unpickle()
label_names = load_label_names()

# print(labels[1])

plt.figure(figsize=(8, 8))
for i in range(25):
    plt.subplot(5, 5, i + 1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(features[i], cmap=plt.cm.binary)
#         print(labels[0][i])

    nom = label_names[labels[0][i]]
    plt.title(nom)

In [ ]:
def unpickle_2classes(label_ids: list) :
    for file in os.listdir("datataset") :
        if file == "batches.meta" :
            continue
        file_name = "datataset/" + file
        with open(file_name, 'rb') as fo :
            dict = pickle.load(fo, encoding='latin1')

        wanted = [i for i, v in enumerate(dict['labels']) if v in label_ids]
        data = np.array([dict['data'][x] for x in wanted])
        data = data.reshape((len(data), 3, 32, 32)).transpose(0, 2, 3, 1)
        label = [dict['labels'][x] for x in wanted]

        return data, label

In [ ]:
def read_data(label_ids: list):
    all_desired_data = []
    all_desired_label = []

    for file in os.listdir('datataset'):
        file_name = "datataset/" + file
        if file == "batches.meta":
            continue
        with open(file_name, 'rb') as fo:
            data_dict = pickle.load(fo, encoding='bytes')

        desired_data_ids = [i for i, v in enumerate(data_dict[b'labels']) if v in label_ids]

        desired_data = [data_dict[b'data'][x] for x in desired_data_ids]
        desired_label = [data_dict[b'labels'][x] for x in desired_data_ids]
        all_desired_data += desired_data
        all_desired_label += desired_label

    all_desired_data = np.array(all_desired_data)
    all_desired_data = all_desired_data.reshape((len(all_desired_data), 3, 32, 32)).transpose(0, 2, 3, 1)


    return all_desired_data, all_desired_label

label_ids = [2, 5]
features, labels = read_data(label_ids=label_ids)

In [ ]:
features, labels = read_data(label_ids=label_ids)
# features, labels = unpickle()
label_names = load_label_names()

plt.figure(figsize=(8, 8))
for i in range(25):
    plt.subplot(5, 5, i + 1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(features[i], cmap=plt.cm.binary)
#         print(labels[0][i])

    nom = label_names[labels[i]]
    plt.title(nom)

In [ ]:
def changer_name(labels) :
    for i in range(len(labels)) :
    #     print(i)
        if labels[i] == 2 :
    #         print("oui")
            labels[i] = 0 #oiseau devient 0
        elif labels[i] == 5 :
            labels[i] = 1 #chien devient 1
    return labels

In [ ]:
labels = changer_name(labels)

plt.figure(figsize=(8, 8))
for i in range(25):
    plt.subplot(5, 5, i + 1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(features[i], cmap=plt.cm.binary)
#     print(labels[i])
    plt.title(labels[i])

In [ ]:
def normalize(x):
    min_val = np.min(x)
    max_val = np.max(x)
    x = (x-min_val) / (max_val-min_val)
    x = np.array(x)
    return x

In [ ]:
def split_data(x, y):

    x_train_validate, x_test, y_train_validate, y_test = train_test_split(x, y, test_size=0.15, random_state=0, stratify=y)
    x_train, x_validate, y_train, y_validate = train_test_split(x_train_validate, y_train_validate, test_size=(15*len(x))/(100*len(x_train_validate)+1), random_state=0, stratify=y_train_validate)
    return [x_train, y_train], [x_test, y_test], [x_validate, y_validate]


In [ ]:
def preparation(features, labels, label_ids: list):

    features, labels = read_data(label_ids=label_ids)
#     labels = changer_name(labels)
    train_origin, test_origin, validate_origin = split_data(features, labels)

    x_train = normalize(train_origin[0])
    x_test = normalize(test_origin[0])
    x_validate = normalize(validate_origin[0])

#     y_train = keras.utils.to_categorical(train_origin[1], 1)
#     y_test = keras.utils.to_categorical(test_origin[1], 1)
#     y_validate = keras.utils.to_categorical(validate_origin[1], 1)

    y_train = np.array(train_origin[1])
    y_test = np.array(test_origin[1])
    y_validate = np.array(validate_origin[1])

    nsamples = y_train.shape
    y_train = y_train.reshape(int(nsamples[0]),1)

    nsamples = y_test.shape
    y_test = y_test.reshape(int(nsamples[0]),1)

    nsamples = y_validate.shape
    y_validate = y_validate.reshape(int(nsamples[0]),1)

    print('shape X train : ', x_train.shape)
    print('shape X test : ', x_test.shape)
    print('shape X validate : ', x_validate.shape)
    print('shape Y train : ', y_train.shape)
    print('shape Y test : ', y_test.shape)
    print('shape Y validate : ', y_validate.shape)

    return [x_train, y_train], [x_test, y_test], [x_validate, y_validate]

In [ ]:
train, test, validate = preparation(features, labels, label_ids = [2, 5])

In [ ]:
train, test, validate = preparation(features, labels, label_ids = [2, 5])
num_classes = 6

y_train = keras.utils.to_categorical(train[1], num_classes)
y_test = keras.utils.to_categorical(test[1], num_classes)
y_validate = keras.utils.to_categorical(validate[1], num_classes)

x_train = train[0]
x_test = test[0]
x_validate = validate[0]

In [ ]:
len

In [ ]:
def pre_graph(data) :
    features_sorted = []
    importance_sorted = []
    oiseau = 0
    chien = 0
    for i in data[1] :
        if i[0] == 2 :
            oiseau += 1
        else :
            chien += 1
    importance_sorted = [oiseau, chien]
    features_sorted = ["oiseau", "chien"]

    return importance_sorted, features_sorted

In [ ]:
c = ['black', 'orange']

In [ ]:
importance_sorted, features_sorted = pre_graph(train)
p2 = plt.bar(range(len(importance_sorted)), importance_sorted, color = c)
plt.bar_label(p2)
plt.title("Distribution des train data : 70%")
plt.xticks(range(len(importance_sorted)), features_sorted)

In [ ]:
importance_sorted, features_sorted = pre_graph(test)
p3 = plt.bar(range(len(importance_sorted)), importance_sorted, color = c)
plt.bar_label(p3)
plt.title("Distribution des test data : 15%")
plt.xticks(range(len(importance_sorted)), features_sorted)

In [ ]:
# plt.figsize=(20,10)
importance_sorted, features_sorted = pre_graph(validate)
p4 = plt.bar(range(len(importance_sorted)), importance_sorted, color = c)
plt.bar_label(p4)
plt.title("Distribution pour les validate data : 15%")
plt.xticks(range(len(importance_sorted)), features_sorted)

## Regression linéaire

In [ ]:
def mettre_list (data) :
    listes = []
    for i in range(len(data)) :
        listes.append(int(data[i]))

    return listes

def changer_name(labels) :
    for i in range(len(labels)) :
    #     print(i)
        if labels[i] == 2 :
    #         print("oui")
            labels[i] = 0 #oiseau devient 0
        elif labels[i] == 5 :
            labels[i] = 1 #chien devient 1
    return labels

In [ ]:
x_train = train[0].reshape(train[0].shape[0], 32*32*3)
x_test = test[0].reshape(test[0].shape[0], 32*32*3)
y_train = mettre_list(train[1])
y_test = mettre_list(test[1])
# print(y_train)
y_train = changer_name(y_train)
y_test = changer_name(y_test)

start_time = time.time()

regressor = LinearRegression()
hist2 = regressor.fit(x_train, y_train)
end_time = time.time()

print(regressor.intercept_)
pd.DataFrame(regressor.coef_)
print(pd.DataFrame(regressor.coef_))

elapsed_time_rl = end_time - start_time
elapsed_time_rl = round(elapsed_time_rl, 3)
print("Elapsed time:", elapsed_time_rl)


In [ ]:
y_pred = regressor.predict(x_test)
y_pred = [np.round(x) for x in y_pred]


for i in range(len(y_pred)) :
    if y_pred[i] == -1.0 :
        y_pred[i] = 1
    elif y_pred[i] == -0.0 :
        y_pred[i] = 0

df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
df

In [ ]:
# print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
# print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
# print("Explain variance score =", metrics.explained_variance_score(y_test, y_pred))
print("R2 score =", metrics.r2_score(y_test, y_pred))

In [ ]:
metrics.accuracy_score(y_test, y_pred)

rl = metrics.accuracy_score(y_test, y_pred)

rl

# Knn

In [ ]:
def get_accuracy(predictions, actual):
    difference = predictions - actual
    number_predictions = predictions.shape[0]
    number_incorrect = np.count_nonzero(difference)
    number_correct = number_predictions - number_incorrect
    accuracy = number_correct / number_predictions
    return accuracy

In [ ]:
classifier = KNeighborsClassifier(n_neighbors=5)
accuracies = []
kf = KFold(n_splits=5, shuffle=True)
x_train = train[0].reshape(train[0].shape[0], 32*32*3)
x_test = test[0].reshape(test[0].shape[0], 32*32*3)
y_train = mettre_list(train[1])
y_test = mettre_list(test[1])
# print(y_train)
y_train = changer_name(y_train)
y_test = changer_name(y_test)

start_time = time.time()

classifier.fit(x_train, y_train)
y_predict = classifier.predict(x_test)
accuracy = get_accuracy(y_predict, y_test)
accuracies.append(accuracy)
end_time = time.time()

mean_accuracy = np.array(accuracies).mean()
mean_accuracy = round(mean_accuracy, 3)
print("Mean accuracy:", mean_accuracy)

knn = mean_accuracy

In [ ]:
elapsed_time_knn = end_time - start_time
elapsed_time_knn = round(elapsed_time_knn, 3)
print("Elapsed time:", elapsed_time_knn)

## CNN


In [ ]:
train, test, validate = preparation(features, labels, label_ids = [2, 5])
num_classes = 6

y_train = keras.utils.to_categorical(train[1], num_classes)
y_test = keras.utils.to_categorical(test[1], num_classes)
y_validate = keras.utils.to_categorical(validate[1], num_classes)

x_train = train[0]
x_test = test[0]
x_validate = validate[0]

In [ ]:
def covnet(baseMapNum = 64, weight_decay = 1e-4):
    keras.backend.clear_session()
    model = Sequential()
    model.add(Conv2D(baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay), input_shape=x_train.shape[1:]))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.2))

    model.add(Conv2D(2*baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(2*baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.3))

    model.add(Conv2D(4*baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(4*baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.45))

    model.add(Flatten())
    model.add(Dense(6, activation='softmax'))

    model.summary()

    return model

In [ ]:
model = covnet()

In [ ]:
datagen = ImageDataGenerator(
    featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    zoom_range=[0.8,1.4],
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.1,
    horizontal_flip=True,
    vertical_flip=False
    )
datagen.fit(train[0])

In [ ]:
optz = keras.optimizers.RMSprop(learning_rate=0.0007,decay=1e-6)
batch_size = 64
steps_epoch=2*x_train.shape[0]
epochs = 30

model.compile(loss='categorical_crossentropy', optimizer=optz, metrics=['accuracy'])
callback = [keras.callbacks.EarlyStopping(monitor="val_loss", mode="min", patience=4, restore_best_weights=True)]

start_time = time.time()

# Fit the model on the batches generated by datagen.flow().
hist1 = model.fit(datagen.flow(x_train, y_train, batch_size=batch_size),
                    validation_data=(x_validate, y_validate),
                    epochs=epochs, verbose=0, callbacks=callback)

end_time = time.time()

model.save_weights('cifar10_ep{}.h5'.format(len(hist1.history['loss'])))


In [ ]:
elapsed_time_cnn = end_time - start_time
elapsed_time_cnn = round(elapsed_time_cnn, 3)
print("Elapsed time:", elapsed_time_cnn)

In [ ]:
def plot_model_result(model_fit):
    plt.subplot(2, 1, 1)
    plt.plot(model_fit.history['accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train'], loc='lower right')

    plt.subplot(2, 1, 2)
    plt.plot(model_fit.history['loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train'], loc='upper right')
    plt.tight_layout()

In [ ]:
hist1.history['accuracy']

In [ ]:
plot_model_result(hist1)

In [ ]:
acc = hist1.history['accuracy']
print("Taux de réussite de : ", max(acc))
cnn = max(acc)

# Comparaison

In [ ]:
# knn
# rl = 0.5
# cnn = 0.80
final = []

final = np.array([["cnn", cnn], ["regression linéaire", rl], ["knn", knn]])
df = pd.DataFrame(final, columns=['Model', 'Accuracy'])
df = df.sort_values(by="Accuracy", ascending=True)
df['Accuracy']=df['Accuracy'].astype(float)
ax = df.plot.barh(x='Model', y='Accuracy')
ax.set_xscale('log')
ax.bar_label(ax.containers[0])
plt.title("Comparaison des taux de précisions des modèles")
plt.show()

In [ ]:
final = []
final = np.array([["cnn", elapsed_time_cnn], ["regression linéaire", elapsed_time_rl], ["knn", elapsed_time_knn]])
df = pd.DataFrame(final, columns=['Model', 'Temps'])
df = df.sort_values(by="Temps", ascending=True)
df['Temps']=df['Temps'].astype(float)
ax = df.plot.barh(x='Model', y='Temps')
ax.set_xscale('log')
ax.bar_label(ax.containers[0])
plt.title("Comparaison des temps d'exécution des modèles (en secondes)")
plt.show()

In [ ]:
def plot_confusion_matrix(model_to_plot, test_data):
    y_pred = model_to_plot.predict(test_data[0])
    y_pred_label = np.argmax(y_pred, 1)
    y_test_label = list(test_data[1])



    mat = confusion_matrix(y_test_label, y_pred_label)
    cmd = ConfusionMatrixDisplay(confusion_matrix=mat.T)
    fig, ax1 = plt.subplots(figsize=(8, 8))
    cmd.plot(cmap=plt.cm.Blues, ax=ax1)

    return y_pred_label, y_test_label

In [ ]:
Y_pred, Y_test = plot_confusion_matrix(model, test)

In [ ]:
def display_errors(label_pred, label_test, data_test):
    label_test = np.argmax(label_test, 1)
    label_test = label_test
    errors = (label_pred - label_test != 0)
    Y_pred_errors_class = label_pred[errors]  # [1, 2, 3, 5, 9, ....]
    Y_test_errors_class = label_test[errors]  # [2, 2, 3, 5, 9, ....]
    X_test_errors = data_test[errors]
    fig, ax3 = plt.subplots(1, 1, figsize=(10, 7))

    error_index = np.where(Y_test_errors_class == 2)
    error_index = np.random.choice(error_index[0], size=1)
#     print(X_test_errors[error_index][0])
    ax3.imshow((X_test_errors[error_index]).reshape((32, 32, 3)), cmap=plt.cm.binary)
    ax3.set_title("Predicted label :{}\nTrue label :{}".format(Y_pred_errors_class[error_index], Y_test_errors_class[error_index]))

#     print(Y_test_errors_class)
#     for row in range(nrows):
#         for col in range(ncols):
#             error_index = np.where(Y_test_errors_class == ncols * row + col)
#             print(error_index)
#             if error_index and len(error_index[0]) != 0:
#                 error_index = np.random.choice(error_index[0], size=1)
#             else:
#                 continue
#             ax3[row, col].imshow((X_test_errors[error_index]).reshape((32, 32)), cmap=plt.cm.binary)
#             ax3[row, col].set_title("Predicted label :{}\nTrue label :{}".format(Y_pred_errors_class[error_index], Y_test_errors_class[error_index]))
#             n += 1

display_errors(Y_pred, y_test, x_test)

In [ ]:
def display_errors5(label_pred, label_test, data_test):
    label_test = np.argmax(label_test, 1)
    label_test = label_test
    errors = (label_pred - label_test != 0)
    Y_pred_errors_class = label_pred[errors]  # [1, 2, 3, 5, 9, ....]
    Y_test_errors_class = label_test[errors]  # [2, 2, 3, 5, 9, ....]
    X_test_errors = data_test[errors]
    fig, ax3 = plt.subplots(1, 1, figsize=(10, 7))

    error_index = np.where(Y_test_errors_class == 5)
    error_index = np.random.choice(error_index[0], size=1)
#     print(X_test_errors[error_index][0])
    ax3.imshow((X_test_errors[error_index]).reshape((32, 32, 3)), cmap=plt.cm.binary)
    ax3.set_title("Predicted label :{}\nTrue label :{}".format(Y_pred_errors_class[error_index], Y_test_errors_class[error_index]))


display_errors5(Y_pred, y_test, x_test)